In [ ]:
import os
import json
from typing import List

import pandas


def collect_stat(benchmark_result_dir:str, csv_path=None):
    if csv_path is None:
        csv_path = os.path.join(benchmark_result_dir, "summary.csv")
        
    all_data = []
    # Loop through the items in the directory
    for item in os.listdir(benchmark_result_dir):
        subdir_full_path = os.path.join(benchmark_result_dir, item)
        
        # Check if the item is a subdirectory
        if not (item.startswith('reqrate') and os.path.isdir(subdir_full_path)):
            print(f'skipping {item}')
            continue
        
        result_json_file = os.path.join(subdir_full_path, 'benchmark.json')
        if not os.path.isfile(result_json_file):
            print(f'skipping {item}')
            continue
        
        # Parse dir name to configs 
        configs = {}
        pairs = item.split('-')
        for pair in pairs:
            key, value = pair.split('_')
            try:
                value = float(value)
            except ValueError:
                pass
            # Add the key-value pair to the dictionary
            configs[key] = value
            
        # Read the benchmark result
        with open(result_json_file) as file:
            metrics = json.load(file)
            
        all_data.append({**configs, **metrics})
        
    df = pandas.DataFrame(all_data)
    df.to_csv(csv_path, index=False)
    
    return csv_path

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


def plot_figure(csv_path:str,
                prefixlen_plot=2048,
                metric_plot='avg_per_output_token_latency',
                reqrate_plot=None,
                x_lim=None,
                y_lim=None,
                save_name=None,
                title=None):
    # metric_plot = 'throughput'
    line_styles = ['-o', '-D', '->']
    df = pd.read_csv(csv_path)
    condition = (df['prefixlen']==prefixlen_plot)
    condition_dict = { 
        'vLLM-RA (ours)': condition & (df['relay']=='true'),
        'vLLM+': condition & (df['relay']=='promptcache'),  
        'vLLM': condition & (df['relay']=='false') 
    }

    data_dict = {}
    for name, cond in condition_dict.items():
        # print(df)
        # print(df['reqrate'])
        view = df[cond].sort_values('reqrate')
        x = view['reqrate'].to_numpy(dtype=np.float32)
        y = view[metric_plot].to_numpy(dtype=np.float32)
        # print(x)
        if reqrate_plot is not None:
            mask = np.in1d(x, reqrate_plot)
            x = x[mask]
            y = y[mask]
        # print(mask)
        data_dict[name] = (x, y)

    # try:
    #     plt_context = plt.style.context(['light', 'grid'])
    #     with plt_context:
    #         pass
    # except:
    #     import contextlib
    #     plt_context = contextlib.nullcontext()
    # plt_context = plt.style.context(['light', 'grid'])
    plt_context = plt.style.context(['grid'])

    with plt_context:
        figure, ax = plt.subplots()
        for idx, (label, data) in enumerate(data_dict.items()):
            x, y = data
            ax.plot(x, y, line_styles[idx], label=label)
        if y_lim:
            ax.set_ylim(y_lim)
        if x_lim:
            ax.set_xlim(x_lim)
        # ax.set_ylabel('Normalized latency (s/token)')
        # ax.set_xlabel('Request Rate (req/s)')
        if title:
            ax.set_title(title)
            # ax.set_title(f'System prompt length = {prefixlen_plot}')
        # ax.legend()
    plt.tight_layout()
    save_dir = os.path.dirname(csv_path)
    if save_name:
        save_path = os.path.join(save_dir, save_name)
        plt.savefig(save_path)
    plt.show()

In [ ]:
exp_root = '../outputs/interactive_bench_sharegpt'
csv_root = '../stat/interactive_bench_sharegpt'
collect = False

os.makedirs(csv_root, exist_ok=True)

figsize=(8, 6)
plt.rcParams.update({'font.size': 16,
                     'font.weight': 'bold',
                     'lines.markersize': 10,
                     'lines.linewidth': 2,
                     'font.family': 'serif',
                     'font.serif': 'Times New Roman',
                     'axes.linewidth': 2
                     })
dpi = 1000

if collect:
    for gpu in os.listdir(exp_root):
        gpu_dir = os.path.join(exp_root, gpu)
        for model in os.listdir(gpu_dir):
            benchmark_result_dir = os.path.join(gpu_dir, model)
            csv_path = os.path.join(csv_root, f'{gpu}.{model}.csv')
            collect_stat(benchmark_result_dir, csv_path=csv_path)

for file in os.listdir(csv_root):
    if file.endswith('.csv'):
        gpu, model, _ = file.split('.')
        csv_path = os.path.join(csv_root,file)
        df = pd.read_csv(csv_path)
        prefixlens = df['prefixlen'].unique()
        for prefixlen_plot in prefixlens:
            print(gpu, model)
            save_name = f'sharegpt.interactive.prefixlen{prefixlen_plot}.{gpu}.{model}.svg'
            plot_figure(csv_path, int(prefixlen_plot), y_lim=(0.0, 1.0), save_name=save_name)

In [ ]:
for file in os.listdir(csv_root):
    if file.endswith('.csv'):
        gpu, model, _ = file.split('.')
        csv_path = os.path.join(csv_root,file)
        df = pd.read_csv(csv_path)
        prefixlens = df['prefixlen'].unique()
        if gpu == 'NVIDIA-A40' and model == 'Llama-2-7b-hf':
            for prefixlen_plot in prefixlens:
                save_name = f'throuput.sharegpt.interactive.prefixlen{prefixlen_plot}.{gpu}.{model}.svg'
                plot_figure(csv_path, int(prefixlen_plot), metric_plot='throughput', save_name=save_name)